In [2]:
import importlib

import numpy as np
import pandas as pd

import pysam
import pyBigWig

from Modules import utils, plot_utils

In [3]:
def get_all_bowtie2_pos_by_query(bamfile: str, paired: bool):
    with pysam.AlignmentFile(bamfile, "rb") as f:
        pos = {}
        for read in f.fetch():
            rname = read.query_name
            chr_id = read.reference_name
            start = read.reference_start
            bt2_align_val = read.tags[0][1]
            if rname not in pos:
                pos[rname] = {}
            if paired:
                nstart = read.next_reference_start
                tlen = abs(read.template_length)
                if not read.is_reverse:
                    stop = start + tlen
                    if (chr_id, start, stop) not in pos[rname]:
                        pos[rname][(chr_id, start, stop)] = []
                    pos[rname][(chr_id, start, stop)].append(bt2_align_val)
                else:
                    nstop = nstart + tlen
                    pos[rname][(chr_id, nstart, nstop)].append(bt2_align_val)
            else:
                qlen = read.query_alignment_length
                stop = start + qlen
                pos[rname][(chr_id, start, stop)] = bt2_align_val
    return pos

In [4]:
# Choose one of the following
bamfile = "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/amorces_qPCR_paired_max200.sorted.bam"
paired = True
extra_suffix = "2"

# bamfile = "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/RNAg-Alu31Rev.sorted.bam"
# paired = False
# extra_suffix = ''

# bamfile = "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/RNAg-Alu31Rev-bis.sorted.bam"
# paired = False
# extra_suffix = ''

In [6]:
# Get matched pairs with alignment score from the bamfile
pos = get_all_bowtie2_pos_by_query(bamfile, paired=paired)
# Write bed files with exact matches and partial matches (i.e. all matches)
print(len(pos))
for repeat_element, positions in pos.items():
    partial_positions = list(positions.keys())
    exact_positions = [
        pos
        for pos, bt2_align_val in positions.items()
        if np.all(np.array(bt2_align_val) == 0)
    ]
    print(repeat_element, len(partial_positions), len(exact_positions))
    for bam_list, suffix in zip(
        [partial_positions, exact_positions], ["partial", "exact"]
    ):
        with open(
            f"/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_{repeat_element}_positions_{suffix}{extra_suffix}.bed",
            "w",
        ) as f:
            for chr_id, start, stop in bam_list:
                f.write(f"{chr_id}\t{start}\t{stop}\n")

3
Alu 70858 5928
Alpha 3071 2339
D4Z1 1013 920


In [10]:
# Check results by counting exact matches by chromosome
bedfiles = {
    "D4Z1": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_D4Z1_positions_exact2.bed",
    "Alpha": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_Alpha_positions_exact2.bed",
    "Alu": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_Alu_positions_exact2.bed",
    "RNAg-Alu31Rev": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_RNAg-Alu31Rev_positions_exact.bed",
    "RNAg-Alu31Rev-bis": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_RNAg-Alu31Rev-bis_positions_exact.bed",
}
for repid, bedfile in bedfiles.items():
    print(repid)
    df = pd.read_csv(bedfile, sep="\t", header=None)
    print(df.groupby(0).count())

D4Z1
0    924
1    924
2    924
dtype: int64
Alpha
0    2348
1    2348
2    2348
dtype: int64
Alu
0    5998
1    5998
2    5998
dtype: int64


Count bigWig file signal over identified repeats

In [ ]:
# Choose appropriate bigwig signals, bed repeat intervals and signal quantiles (computed from bigwig_stats.ipynb)
bwfiles = {
    "Ctrl": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C44_trimmed_run2_paired_T2T.bw",
    "IP1": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/D1145C43_trimmed_run2_paired_T2T.bw",
    "IP2": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/HN00205099_RawFASTQ_RPE1_WTH3K9me3_run2_paired_T2T.bw",
}
bedfiles = {
    "D4Z1": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_D4Z1_positions_exact2.bed",
    "Alpha": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_Alpha_positions_exact2.bed",
    "Alu": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_Alu_positions_exact2.bed",
    "RNAg-Alu31Rev": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_RNAg-Alu31Rev_positions_exact.bed",
    "RNAg-Alu31Rev-bis": "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_RNAg-Alu31Rev-bis_positions_exact.bed",
}
quantiles = pd.read_csv(
    "/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/quantiles_run2.csv",
    index_col=0,
)
quantiles

In [2]:
suffix = "_run2"
# Choose options:

# Count on 300bp centered on interval (default is only interval)
# suffix += "_300bp"

# Count using thresholded signal at 99e centile
# suffix += "_clipq0.99"

suffix

'_run2_300bp_clipq0.99'

In [ ]:
# Count total signal by interval
tot = {}
for file_id, bwfile in bwfiles.items():
    tot[file_id] = {}
    with pyBigWig.open(bwfile) as bw:
        for rep_id, bedfile in bedfiles.items():
            tot[file_id][rep_id] = {}
            with open(bedfile) as f:
                for line in f:
                    chr_id, start, stop, *_ = line.split()
                    start, stop = int(start), int(stop)
                    # Taking 300bp centered on bed interval
                    if "_300bp" in suffix:
                        mid = (stop + start) // 2
                        start, stop = mid - 150, mid + 150
                        stop = min(bw.chroms(chr_id) - 1, stop)
                    #################################
                    interval_sum = bw.values(chr_id, start, stop, numpy=True)
                    # Cliping signal before summing
                    if "_clipq0.99" in suffix:
                        interval_sum = np.clip(
                            interval_sum,
                            a_min=None,
                            a_max=quantiles.loc[bwfile, "quantile_0.99"],
                        )
                    #################################
                    tot[file_id][rep_id][(chr_id, start, stop)] = np.sum(interval_sum)
# Index by rep_id first and make a list of counts by file
frag_counts = {}
for rep_id in bedfiles:
    frag_counts[rep_id] = {}
    for file_id, file_tot in tot.items():
        for pos, val in file_tot[rep_id].items():
            if pos not in frag_counts[rep_id]:
                frag_counts[rep_id][pos] = []
            frag_counts[rep_id][pos].append(int(val))
frag_counts
# Store result in bed files
resbedfiles = {
    "D4Z1": f"/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_D4Z1_positions_exact{suffix}_frag_counts.bed",
    "Alpha": f"/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_Alpha_positions_exact{suffix}_frag_counts.bed",
    "Alu": f"/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_Alu_positions_exact{suffix}_frag_counts.bed",
    "RNAg-Alu31Rev": f"/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_RNAg-Alu31Rev_positions_exact{suffix}_frag_counts.bed",
    "RNAg-Alu31Rev-bis": f"/home/alex/shared_folder/Judith-H3K9me3/results/alignments/T2T-CHM13v2.0/T2T_RNAg-Alu31Rev-bis_positions_exact{suffix}_frag_counts.bed",
}
for rep_id, resbedfile in resbedfiles.items():
    resbedfile = utils.safe_filename(resbedfile)
    with open(resbedfile, "w") as f:
        for (chr_id, start, stop), (ctrl, ip1, ip2) in frag_counts[rep_id].items():
            f.write(f"{chr_id}\t{start}\t{stop}\t{ctrl}\t{ip1}\t{ip2}\n")